In [111]:
import sqlite3
import pandas as pd
import plotly.express as px

# Connect to SQLite database
connect = sqlite3.connect('data/kbo_database.db')
  
# Create a cursor object
cur = connect.cursor()


In [112]:
q = """
SELECT 
    strftime('%Y', DATE(substr(StartDate, 7, 4) || '-' || substr(StartDate, 4, 2) || '-' || substr(StartDate, 1, 2))) AS year,
    ((CAST(substr(StartDate, 4, 2) AS INTEGER) - 1) / 3 + 1) AS quarter,
    description_nl,
    COUNT(*) AS count
FROM enterprise AS e
JOIN JuridicalForm AS j ON e.JuridicalForm = j.code
WHERE DATE(substr(StartDate, 7, 4) || '-' || substr(StartDate, 4, 2) || '-' || substr(StartDate, 1, 2)) >= '2015-01-01'
AND NOT (
    strftime('%Y', DATE(substr(StartDate, 7, 4) || '-' || substr(StartDate, 4, 2) || '-' || substr(StartDate, 1, 2))) = '2025'
    AND ((CAST(substr(StartDate, 4, 2) AS INTEGER) - 1) / 3 + 1) = 3
  )
GROUP BY year, quarter, description_nl
ORDER BY year ASC, quarter ASC, count DESC
"""
df_form_quarterly = pd.read_sql_query(q, connect)

top_10_forms = (
    df_form_quarterly
    .groupby("description_nl")["count"]
    .sum()
    .nlargest(10)
    .index
)

df_form_quarterly['description_nl'] = df_form_quarterly['description_nl'].where(
    df_form_quarterly['description_nl'].isin(top_10_forms),
    other='Other'
)

df_grouped = df_form_quarterly.groupby(
    ['year', 'quarter', 'description_nl'], as_index=False
)['count'].sum()

df_grouped['year_quarter'] = df_grouped['year'] + '-Q' + df_grouped['quarter'].astype(int).astype(str)
df_grouped = df_grouped.pivot(index='year_quarter', columns='description_nl', values='count').fillna(0)


fig = px.line(
    df_grouped,
    title='Juridical Form Trends by Quarter (since 2015)',
    labels={'value': 'Number of Enterprises', 'year_quarter': 'Quarter'},
)

fig.update_layout(

    yaxis_title='Number of Enterprises',
    legend_title_text='Juridical Form',
    legend_title_font=dict(size=18),
    legend_font=dict(size=16),
    legend=dict(
        orientation="h",      
        yanchor="top",        
        y=-0.3,                
        xanchor="center",     
        x=0.5                 
    ),
    width=1200,
    height = 600
    
)

fig.show()






In [113]:
q ="""SELECT 
    g."Région name (Dutch)" AS region,
    COUNT(*) AS activity_count   
FROM activity_lable_cleaned AS a
JOIN address AS ad ON a.EntityNumber = ad.EntityNumber
JOIN geo_data AS g ON ad.Zipcode = g."Post code"
WHERE ad.CountryNL IS NULL 
AND Classification = 'MAIN'
GROUP BY region
ORDER BY activity_count DESC;
"""
df_overall = pd.read_sql_query(q, connect)

region_mapping = {
    "Vlaams Gewest": "Flemish Region",
    "Waals Gewest": "Walloon Region",
    "Brussels Hoofdstedelijk Gewest": "Brussels Capital Region"
}

df_overall['region'] = df_overall['region'].replace(region_mapping)


print(df_overall.head(10))




                    region  activity_count
0           Flemish Region         3227170
1           Walloon Region         1437103
2  Brussels Capital Region          803278


In [114]:


fig = px.pie(
    df_overall, 
    names='region', 
    values='activity_count',
    title='Share of Economic Activities per Region',
    color='region',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_traces(
    textinfo='percent+label',
    texttemplate='%{percent:.1%}',
    hovertemplate='%{label}: %{value} activities<br>(%{percent:.1%})'
)


fig.update_layout(
    title_x=0.8, 
    margin=dict(t=40, b=20, l=20, r=20),  
    showlegend=True,
    legend_title_text='Region',
    legend=dict(
        orientation="v",     
        yanchor="middle",    
        y=0.5,               
        xanchor="left",     
        x=0.7,          
        font=dict(size=14)
    ),
    height=400,   
  
)



fig.show()


In [115]:
q = """SELECT 
    g."Région name (Dutch)" AS region,
    ad.Zipcode,
    a.nace_section,
    n.description,
    COUNT(*) AS company_count
FROM activity_lable_cleaned AS a
JOIN address AS ad ON a.EntityNumber = ad.EntityNumber
JOIN geo_data AS g ON ad.Zipcode = g."Post code"
JOIN nace_section AS n ON a.nace_section = n.section
WHERE CountryNL IS NULL
AND Classification = 'MAIN'
GROUP BY g."Région name (Dutch)", a.nace_section
ORDER BY g."Région name (Dutch)", company_count DESC;
"""

df = pd.read_sql_query(q, connect)

df['region'] = df['region'].replace(region_mapping)

sector_totals = df.groupby('description')['company_count'].sum().reset_index()
sector_totals = sector_totals.rename(columns={'company_count': 'total_company_count'})

df = df.merge(sector_totals, on='description')

df = df.sort_values(by=['region', 'total_company_count', 'company_count'], ascending=[True, False, False])

print(df.head(10))



                     region Zipcode nace_section  \
0   Brussels Capital Region    1170            M   
1   Brussels Capital Region    1180            G   
2   Brussels Capital Region    1000            N   
10  Brussels Capital Region    1000            C   
5   Brussels Capital Region    1000            F   
3   Brussels Capital Region    1080            H   
6   Brussels Capital Region    1180            S   
4   Brussels Capital Region    1082            J   
11  Brussels Capital Region    1000            Q   
7   Brussels Capital Region    1180            R   

                                          description  company_count  \
0   Professional, scientific and technical activities         149902   
1                          Wholesale and retail trade         104930   
2       Administrative and support service activities          86112   
10                                      Manufacturing          38357   
5                                        Construction          5082

In [116]:
fig = px.bar(
    df,
    x='description',          
    y='company_count',
    color='region',             
    barmode='stack',         
    labels={
        'nace_section': 'NACE Section',
        'company_count': 'Number of Economic Activities',
        'region': 'Region'
    },
    title='Total Economic Activities per Sector by Region (Stacked)',
    category_orders={
        'region': ['Flanders', 'Wallonia', 'Brussels']
    },
    width=1000,
    height=600
)

fig.show()


In [117]:

df_sorted = df.sort_values(['region', 'company_count'], ascending=[True, False])
top5_per_region = df_sorted.groupby('region').head(5).reset_index(drop=True)
region_totals = df.groupby('region')['company_count'].sum().reset_index()
region_totals.rename(columns={'company_count': 'region_total'}, inplace=True)
top5_per_region = top5_per_region.merge(region_totals, on='region', how='left')
top5_per_region['percent_share_full'] = (top5_per_region['company_count'] / top5_per_region['region_total']) * 100
result_full = top5_per_region[['region', 'description', 'percent_share_full']].copy()
result_full['percent_share_full'] = result_full['percent_share_full'].round(2)
print(result_full)





                     region  \
0   Brussels Capital Region   
1   Brussels Capital Region   
2   Brussels Capital Region   
3   Brussels Capital Region   
4   Brussels Capital Region   
5            Flemish Region   
6            Flemish Region   
7            Flemish Region   
8            Flemish Region   
9            Flemish Region   
10           Walloon Region   
11           Walloon Region   
12           Walloon Region   
13           Walloon Region   
14           Walloon Region   

                                          description  percent_share_full  
0   Professional, scientific and technical activities               18.69  
1                          Wholesale and retail trade               13.09  
2       Administrative and support service activities               10.74  
3                          Transportation and storage                6.91  
4                       Information and communication                6.79  
5   Professional, scientific and technical acti

In [118]:
df_percent = df.groupby(['region', 'description'])['company_count'].sum().reset_index()

df_percent['percent'] = df_percent.groupby('region')['company_count'].transform(
    lambda x: 100 * x / x.sum()
)

top_sectors = df.groupby('description')['company_count'].sum().nlargest(12).index
df_percent['description'] = df_percent['description'].apply(
    lambda x: x if x in top_sectors else 'Other'
)

df_percent_grouped = df_percent.groupby(['region', 'description'])['percent'].sum().reset_index()


colors = px.colors.qualitative.Set3
while len(colors) < len(top_sectors) + 1:
    colors += colors
sector_labels = list(top_sectors) + ['Other']
color_map = {sector: color for sector, color in zip(sector_labels, colors)}
color_map['Other'] = '#cccccc'

fig = px.bar(
    df_percent_grouped,
    x='region',
    y='percent',
    color='description',
    title='Sector Share by Region (Relative %)',
    labels={'percent': 'Percent Share', 'description': 'Sector'},
    text_auto='.1f',
    color_discrete_map=color_map
)

fig.update_layout(
    barmode='stack',
    title_x=0.25,
    yaxis=dict(ticksuffix="%"),
    height=600
)

fig.update_traces(textfont_size=11)
fig.show()



In [119]:

all_sectors = sorted(df['description'].unique())

colors = px.colors.qualitative.Set2
while len(colors) < len(all_sectors):
    colors += colors
color_map = {sector: color for sector, color in zip(all_sectors, colors)}
color_map['Other'] = '#cccccc'

regions = df['region'].unique()

for region in regions:
    df_region = df[df['region'] == region].copy()
    top_10 = df_region.groupby('description')['company_count'].sum().sort_values(ascending=False).head(10).index.tolist()
    df_region['description'] = df_region['description'].apply(
        lambda x: x if x in top_10 else 'Other'
    )

    pie_data = df_region.groupby('description')['company_count'].sum().reset_index()

    fig = px.pie(
        pie_data, 
        names='description', 
        values='company_count',
        title=f'Sector Distribution in {region} (Top 10 + Other)',
        color='description',
        color_discrete_map=color_map
    )

    fig.update_layout(
        legend=dict(
            x=0.05, 
            y=0.5,
            traceorder='normal',
            font=dict(size=12),
            bgcolor='rgba(0,0,0,0)',
            borderwidth=0
        ),
        title_x=0.15
    )

    fig.show()





In [120]:
fig = px.treemap(
    df,
    path=['region', 'description'],
    values='company_count',
    color='region',
    title='Sector Distribution by Region (Treemap)',
    color_discrete_sequence=px.colors.qualitative.Set2,
    width=1000,
    height=600
)

fig.update_traces(
    textinfo='label+value', 
    tiling=dict(pad=5), 
    marker=dict(cornerradius=5),
    textfont_size=12
)

fig.update_layout(
    title=dict(
        text='Sector Distribution by Region (Treemap)',
        x=0.5,
        xanchor='center'
    )
)

fig.show()


In [121]:
q_check_duplicate = """
SELECT EnterpriseNumber, COUNT(*) AS cnt
FROM enterprise
GROUP BY EnterpriseNumber
HAVING COUNT(*) > 1 LIMIT 100"""

df_check_duplicate = pd.read_sql_query(q_check_duplicate, connect)

print(df_check_duplicate.head(10))

Empty DataFrame
Columns: [EnterpriseNumber, cnt]
Index: []


In [122]:
q = """SELECT 
    ad.MunicipalityNL AS Municipality,
    g."Région name (Dutch)" AS Region,
    CAST(substr(g."Geo Point", 1, instr(g."Geo Point", ',') - 1) AS FLOAT) AS latitude,
    CAST(substr(g."Geo Point", instr(g."Geo Point", ',') + 2) AS FLOAT) AS longitude,
    n.description AS sector,
    COUNT(*) AS company_count
FROM activity_lable_cleaned AS a
JOIN address AS ad ON a.EntityNumber = ad.EntityNumber
JOIN geo_data AS g ON ad.Zipcode = g."Post code"
JOIN enterprise AS e ON a.EntityNumber = e.EnterpriseNumber
JOIN JuridicalForm AS j ON e.JuridicalForm = j.code
JOIN nace_section AS n ON a.nace_section = n.section
WHERE a.Classification = 'MAIN'
  AND j.description_nl = 'Besloten Vennootschap'
  AND n.description IN (
    'Professional, scientific and technical activities',
    'Wholesale and retail trade',
    'Administrative and support service activities'
  )
  AND ad.CountryNL IS NULL
GROUP BY Municipality, latitude, longitude, sector
HAVING COUNT(*) > 5
ORDER BY company_count DESC;
"""

df = pd.read_sql_query(q, connect)

print(df.head(10))



  Municipality                          Region   latitude  longitude  \
0    Antwerpen                   Vlaams Gewest  51.248448   4.376134   
1    Antwerpen                   Vlaams Gewest  51.248448   4.376134   
2      Brussel  Brussels Hoofdstedelijk Gewest  50.872983   4.375234   
3      Brussel  Brussels Hoofdstedelijk Gewest  50.822326   4.377066   
4       Elsene  Brussels Hoofdstedelijk Gewest  50.822326   4.377066   
5       Elsene  Brussels Hoofdstedelijk Gewest  50.829476   4.344698   
6        Ukkel  Brussels Hoofdstedelijk Gewest  50.789871   4.361370   
7      Brussel  Brussels Hoofdstedelijk Gewest  50.872983   4.375234   
8      Brussel  Brussels Hoofdstedelijk Gewest  50.853812   4.369279   
9         Gent                   Vlaams Gewest  51.053261   3.720262   

                                              sector  company_count  
0  Professional, scientific and technical activities           3498  
1                         Wholesale and retail trade           2870

In [123]:

fig = px.scatter_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    size='company_count',
    color='sector',
    hover_name='Municipality',
    hover_data={'company_count': True, 'latitude': False, 'longitude': False},
    size_max=30,
    zoom=7,  
    mapbox_style='open-street-map',  
    title='Business Hotspots by Sector and Municipality (Legal form: Besloten Vennootschap)'
)

fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()




/var/folders/6k/0qvxrpnd0rd_bcv5419k929r0000gn/T/ipykernel_6061/123886987.py:1: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [124]:

sectors_to_highlight = ['Professional, scientific and technical activities',
    'Wholesale and retail trade',
    'Administrative and support service activities']

for sector in sectors_to_highlight:
    sector_df = df[df['sector'] == sector]
    fig = px.scatter_map(
        sector_df,
        lat='latitude',
        lon='longitude', 
        size='company_count',
        hover_name='Municipality',
        size_max=40,
        zoom=7,
        map_style='open-street-map',  
        title=f'{sector} Business Clusters'
    )
    fig.show()
    fig.write_image(f"belgium_{sector.lower().replace(' ', '_')}_hotspots.png", 
                   width=800, height=600, scale=2)
    

In [125]:

top_cities = df.groupby(['Municipality', 'Region'])['company_count'].sum().reset_index()
top_cities = top_cities.sort_values('company_count', ascending=False).head(10)
print(top_cities[['Municipality', 'Region', 'company_count']])

    Municipality                          Region  company_count
87       Brussel  Brussels Hoofdstedelijk Gewest          14878
17     Antwerpen                   Vlaams Gewest          12332
143       Elsene  Brussels Hoofdstedelijk Gewest           7332
185         Gent                   Vlaams Gewest           6038
319        Liège                    Waals Gewest           4159
509        Ukkel  Brussels Hoofdstedelijk Gewest           3343
303       Leuven                   Vlaams Gewest           2864
154    Etterbeek  Brussels Hoofdstedelijk Gewest           2624
211      Hasselt                   Vlaams Gewest           2437
85        Brugge                   Vlaams Gewest           2271


In [126]:
import plotly.express as px

fig = px.bar(
    top_cities,
    x='company_count',
    y='Municipality',
    color='Region',
    orientation='h',
    title='Top 10 BV Municipalities – Technical Activities, Trade, Admin Support',
    labels={'company_count': 'Number of Companies', 'Municipality': 'City'},
    height=500,
    width=800,
    color_discrete_sequence=px.colors.qualitative.Set3,
)

fig.update_layout(
    yaxis={'categoryorder':'total ascending'},
    xaxis_title='Number of Companies(BV)',
    yaxis_title='Municipality',
    legend=dict(
        orientation="h",      
        yanchor="top",        
        y=-0.2,                
        xanchor="center",     
        x=0.5                 
    ),
)

fig.show()




In [127]:
q = """SELECT 
    strftime('%Y', DATE(substr(e.StartDate, 7, 4) || '-' || substr(e.StartDate, 4, 2) || '-' || substr(e.StartDate, 1, 2))) AS year,
    n.description AS sector,
    COUNT(*) AS count
FROM enterprise AS e
JOIN activity_lable_cleaned AS a ON e.EnterpriseNumber = a.EntityNumber
JOIN nace_section AS n ON a.nace_section = n.section
JOIN JuridicalForm AS j ON e.JuridicalForm = j.code
WHERE DATE(substr(e.StartDate, 7, 4) || '-' || substr(e.StartDate, 4, 2) || '-' || substr(e.StartDate, 1, 2)) >= '2015-01-01'
  AND a.Classification = 'MAIN'
  AND j.description_nl = 'Besloten Vennootschap'
  AND n.description IN (
    'Professional, scientific and technical activities',
    'Wholesale and retail trade',
    'Administrative and support service activities',
    'Manufacturing',
    'Transportation and storage',
    'Construction'
  )
  AND NOT (
    strftime('%Y', DATE(substr(e.StartDate, 7, 4) || '-' || substr(e.StartDate, 4, 2) || '-' || substr(e.StartDate, 1, 2))) = '2025'

  )
GROUP BY year, sector
ORDER BY year, count DESC;"""


df = pd.read_sql_query(q, connect)

print(df.head(10))


   year                                             sector  count
0  2015  Professional, scientific and technical activities   2676
1  2015                         Wholesale and retail trade   1821
2  2015                                       Construction   1773
3  2015      Administrative and support service activities    736
4  2015                                      Manufacturing    672
5  2015                         Transportation and storage    285
6  2016  Professional, scientific and technical activities   3097
7  2016                                       Construction   1995
8  2016                         Wholesale and retail trade   1994
9  2016      Administrative and support service activities    855


In [128]:
print(df.groupby('sector')['count'].sum().sort_values(ascending=False))


sector
Professional, scientific and technical activities    58799
Construction                                         41222
Wholesale and retail trade                           31090
Administrative and support service activities        16083
Manufacturing                                        12028
Transportation and storage                            8216
Name: count, dtype: int64


In [129]:


df['year'] = df['year'].astype(int)

df_sorted = df.sort_values(['sector', 'year'])
df_sorted['growth_pct'] = df_sorted.groupby('sector')['count'].pct_change() * 100
df_growth = df_sorted.dropna(subset=['growth_pct'])

top_sectors = [
    'Professional, scientific and technical activities',
    'Wholesale and retail trade',
    'Administrative and support service activities'
]

support_sectors = [
    'Manufacturing',
    'Transportation and storage',
    'Construction'
]

df['year'] = df['year'].astype(int)

df_sorted = df.sort_values(['sector', 'year'])
df_sorted['growth_pct'] = df_sorted.groupby('sector')['count'].pct_change() * 100
df_growth = df_sorted.dropna(subset=['growth_pct'])

top_sectors = [
    'Professional, scientific and technical activities',
    'Wholesale and retail trade',
    'Administrative and support service activities'
]

support_sectors = [
    'Manufacturing',
    'Transportation and storage',
    'Construction'
]


fig_top = px.line(df_growth, x='year', y='growth_pct', color='sector',
                  markers=True,
                  title='Year-over-Year Growth – Top Business Sectors (%)',
                  color_discrete_sequence=px.colors.qualitative.Set3)

fig_top.update_layout(
    yaxis_title='growth %',
    legend=dict(
        orientation="h",      
        yanchor="top",        
        y=-0.1,                
        xanchor="center",     
        x=0.5,
        ),
    height= 650)
fig_top.show()





df_top = df_growth[df_growth['sector'].isin(top_sectors)]
fig_top = px.line(df_top, x='year', y='growth_pct', color='sector',
                  markers=True,
                  title='Year-over-Year Growth – Top Business Sectors (%)')
fig_top.show()



In [130]:
df_support = df_growth[df_growth['sector'].isin(support_sectors)]
fig_support = px.line(df_support, x='year', y='growth_pct', color='sector',
                      markers=True,
                      title='🔧 Year-over-Year Growth – Support Sectors (%)')

fig_support.show()

In [131]:
df_2024 = df_growth[df_growth['year'] == 2024]
fig_top = px.bar(
    df_2024,
    x='growth_pct',
    y='sector',
    color='sector',
    text='growth_pct',
    title='2024 Growth – Top Business Sectors (%)',
    orientation='h',
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig_top.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig_top.update_layout(
    xaxis_title='Sector',
    yaxis_title='Growth (%)',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    showlegend=False,

    width=1300,
)

fig_top.show()